# Lab 4: Azure AI Foundry Agent Service - Workflow Pattern

## 개요 (Overview)

이 노트북에서는 **Azure AI Foundry Agent Service**를 활용하여 **Workflow Pattern** 기반의 Multi-Agent 시스템을 배포합니다.

### 🔑 핵심 구현 방식

- **Azure AI Foundry Agent Service**: Lab 3과 동일한 Agent 기반
- **Workflow Pattern**: Router + Executor 패턴으로 오케스트레이션
- **AI 기반 라우팅**: LLM을 사용한 의도 분류
- **Workflow Context**: 메시지 기반 상태 관리

### 💡 Lab 3과의 차이점

| 특성 | Lab 3 (Connected Agent) | Lab 4 (이 노트북) |
|------|------------------------|------------------|
| **Agent 기반** | ✅ Foundry Agent Service | ✅ Foundry Agent Service |
| **워크플로우** | Connected Agent (Handoff) | **Workflow Pattern (Router+Executor)** |
| **라우팅 방식** | `handoff_to_agent()` API | **Router Executor 함수** |
| **실행 흐름** | Main → Handoff → Sub Agent | **Router → Executor → Output** |
| **상태 관리** | Thread 기반 | **Workflow Context 기반** |
| **병렬 실행** | 순차 Handoff | **Orchestrator 병렬 가능** |

> **✅ 공통점**: 두 Lab 모두 **동일한 Azure AI Foundry Agent Service**를 사용합니다.
> 
> **🎯 차이점**: **Agent 오케스트레이션 패턴**이 다릅니다 (Connected Agent vs Workflow Pattern).
> 
> **⚡ 장점**: Workflow Pattern은 복잡한 조건 분기, 병렬 실행, 루프 등 고급 오케스트레이션이 가능합니다.

### 아키텍처 (Architecture)
```
┌────────────────────────────────────────────────────────────┐
│        Agent Framework - Workflow Pattern                 │
│                                                            │
│  ┌──────────────────────────────────────────┐             │
│  │        Router Executor                   │             │
│  │   (AI-based Intent Classification)       │             │
│  └────┬──────┬────────┬────────────┬────────┘             │
│       │      │        │            │                      │
│   ┌───▼──┐ ┌▼───┐  ┌─▼────┐   ┌──▼────────┐             │
│   │ Tool │ │Research│ │General│ │Orchestrator│            │
│   │Exec  │ │Executor│ │Executor│ │Executor  │             │
│   └───┬──┘ └┬───┘  └─┬────┘   └──┬────────┘             │
│       │     │        │            │                      │
│   ┌───▼─────▼────────▼────────────▼──────┐               │
│   │      Workflow Context                │               │
│   │   (Message Passing & Output)         │               │
│   └──────────────────────────────────────┘               │
│                                                            │
│   External Resources:                                     │
│   ┌──────────────┐    ┌────────────────┐                 │
│   │  MCP Server  │    │  Azure AI      │                 │
│   │  (Tools)     │    │  Search (RAG)  │                 │
│   └──────────────┘    └────────────────┘                 │
└────────────────────────────────────────────────────────────┘
```

### 주요 차이점: Foundry Agent vs Agent Framework

| 특성 | Foundry Agent (Lab 3) | Agent Framework (Lab 4) |
|------|----------------------|-------------------------|
| **패턴** | Connected Agent (Handoff) | Workflow Executor Pattern |
| **메시지 흐름** | Thread-based conversation | Workflow Context streaming |
| **Agent 간 통신** | Agent API 호출 (handoff) | Workflow message routing |
| **실행 모델** | Synchronous handoff | Async executor graph |
| **확장성** | Agent 추가 시 handoff 설정 | Executor & Edge 추가 |
| **추적** | Agent Service 기본 추적 | Custom OpenTelemetry |
| **상태 관리** | Thread 상태 유지 | Workflow Context 기반 |
| **복잡도** | 중간 (Azure 관리형) | 낮음 (코드 중심) |

### Workflow Pattern의 장점

1. ✅ **유연한 메시지 라우팅**: 단순한 조건문으로 복잡한 흐름 제어
2. ✅ **비동기 실행**: Parallel execution 지원 (Orchestrator)
3. ✅ **명확한 실행 그래프**: WorkflowBuilder로 시각적 구조 정의
4. ✅ **로컬 개발 용이**: Azure 의존성 최소화, 빠른 테스트
5. ✅ **커스텀 제어**: 세밀한 오류 처리 및 로깅

### Python 모듈 구조 (Python Modules)

```
src/agent_framework/
├── main_agent_workflow.py  - Workflow 정의 및 Executor 구현
├── tool_agent.py           - Tool Agent 클래스 (MCP)
├── research_agent.py       - Research Agent 클래스 (RAG)
├── api_server.py           - FastAPI HTTP 서버
├── masking.py              - 마스킹 유틸리티
└── requirements.txt        - Python 의존성
```

### 학습 목표 (Learning Objectives)

1. ✅ Microsoft Agent Framework Workflow Pattern 이해
2. ✅ Executor 기반 Multi-Agent 구현
3. ✅ Workflow Context를 통한 메시지 라우팅
4. ✅ Parallel execution (Orchestrator) 패턴
5. ✅ Agent Framework 기반 시스템을 ACA에 배포
6. ✅ Foundry Agent와 Agent Framework 비교 분석

## 1. 환경 설정 및 인증 (Setup & Authentication)

### 테넌트 ID 설정 안내

**대부분의 경우**: 테넌트 ID를 지정하지 않아도 됩니다. `tenant_id` 변수를 `"<YOUR_TENANT_ID>"` 또는 `None`으로 두고 실행하세요.

**테넌트 ID가 필요한 경우**:
- ✅ 여러 조직(회사)의 Azure 테넌트에 접근 권한이 있는 경우
- ✅ 특정 조직의 리소스로만 작업해야 하는 경우
- ✅ 로그인 시 "multiple tenants" 관련 오류가 발생하는 경우

**테넌트 ID 확인 방법**:
- Azure Portal → Azure Active Directory → 개요 → 테넌트 ID 복사
- 또는 조직 관리자에게 문의

In [ ]:
import sys, subprocess, os, json
import platform

# 운영체제에 따라 PATH 설정
system = platform.system()
if system == 'Darwin':  # macOS
    extra_paths = '/opt/homebrew/bin:/usr/local/bin'
elif system == 'Linux':  # Linux / Codespaces
    extra_paths = '/usr/local/bin:/usr/bin:/home/codespace/.local/bin'
else:  # Windows
    extra_paths = ''

if extra_paths:
    os.environ['PATH'] = extra_paths + ':' + os.environ.get('PATH', '')

def check(cmd, name):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, timeout=3, env=os.environ)
        print(f"{'✓' if result.returncode == 0 else '✗'} {name}")
    except Exception as e:
        print(f"✗ {name}")

print("=== Prerequisites Check ===")
print(f"✓ Python {sys.version.split()[0]} ({system})")
check("az --version", "Azure CLI")
check("docker --version", "Docker")
print("="*50)

In [ ]:
import subprocess, json

print("=== Azure Authentication ===")
print("ℹ️  인증 상태를 확인하고 필요시 로그인합니다.\n")

# 테넌트 ID를 여기에 입력하세요 (선택사항)
# 예: tenant_id = "16b3c013-d300-468d-ac64-7eda0820b6d3"
tenant_id = "<YOUR_TENANT_ID>"  # 또는 None으로 설정하면 기본 테넌트 사용

# Azure CLI 인증 상태 확인
az_account = subprocess.run("az account show", shell=True, capture_output=True, text=True)

if az_account.returncode == 0:
    account_info = json.loads(az_account.stdout)
    print(f"✅ Azure CLI 인증 완료 (기존 세션 사용)")
    print(f"   구독: {account_info.get('name', 'N/A')}")
    print(f"   테넌트: {account_info.get('tenantId', 'N/A')}")
else:
    print("⚠️  Azure CLI 인증이 필요합니다. 브라우저가 열립니다...")
    # 테넌트 ID가 설정되어 있으면 해당 테넌트로 로그인
    if tenant_id and tenant_id != "<YOUR_TENANT_ID>":
        az_login = subprocess.run(f"az login --tenant {tenant_id}", shell=True)
    else:
        az_login = subprocess.run("az login", shell=True)
    
    if az_login.returncode == 0:
        print("✅ Azure CLI 로그인 완료")
    else:
        raise Exception("❌ Azure CLI 로그인 실패")

print("="*50)

In [ ]:
# 설정 파일 로드
config_path = "config.json"
with open(config_path) as f:
    config = json.load(f)

# 환경 변수 설정
RESOURCE_GROUP = config["resource_group"]
LOCATION = config["location"]
PROJECT_CONNECTION_STRING = config["project_connection_string"]
SEARCH_ENDPOINT = config["search_endpoint"]
SEARCH_INDEX = config["search_index"]
CONTAINER_REGISTRY = config["container_registry_endpoint"]
CONTAINER_ENV_ID = config["container_apps_environment_id"]
MCP_ENDPOINT = config.get("mcp_endpoint", "")

# PROJECT_CONNECTION_STRING을 간단한 형식으로 변환
simple_project_conn = PROJECT_CONNECTION_STRING.split(';')[0] if PROJECT_CONNECTION_STRING else ""

print("=== Configuration Loaded ===")
print(f"Resource Group: {RESOURCE_GROUP}")
print(f"Location: {LOCATION}")
print(f"Search Index: {SEARCH_INDEX}")
print(f"MCP Endpoint: {MCP_ENDPOINT if MCP_ENDPOINT else 'Not configured'}")
print(f"Container Registry: {CONTAINER_REGISTRY}")
print("="*50)

## 2. 필수 패키지 설치 (Install Required Packages)

Azure AI 관련 필수 패키지를 설치합니다. GitHub Codespace에서 실행하는 경우 대부분의 패키지가 이미 설치되어 있을 수 있습니다.

In [ ]:
# 필수 패키지 설치
import subprocess
import sys

packages = [
    "azure-ai-projects",
    "azure-ai-inference",
    "azure-search-documents",
    "azure-identity",
    "openai",
    "python-dotenv",
    "requests",
    "fastapi",
    "uvicorn",
    "httpx"
]

print("=== Installing Required Packages ===\n")

for package in packages:
    print(f"Installing {package}...")
    result = subprocess.run(
        [sys.executable, "-m", "pip", "install", "-q", package],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        print(f"✅ {package} installed")
    else:
        print(f"⚠️  {package} may already be installed or failed to install")

print("\n" + "="*50)
print("✅ Package installation completed!")

## 3. Azure AI Search 키 가져오기 (Get Search Key)

Research Agent가 사용할 Azure AI Search 관리 키를 가져옵니다.

In [ ]:
# AI Search 관리 키 가져오기
search_name = config["search_service_name"]

search_key_cmd = f"""
az search admin-key show \
    --resource-group {RESOURCE_GROUP} \
    --service-name {search_name} \
    --query primaryKey -o tsv
"""

result = subprocess.run(search_key_cmd, shell=True, capture_output=True, text=True)
SEARCH_KEY = result.stdout.strip()

if SEARCH_KEY:
    print(f"✅ Search key retrieved: {SEARCH_KEY[:10]}...")
    os.environ['SEARCH_KEY'] = SEARCH_KEY
else:
    print("❌ Failed to retrieve search key")

## 4. Agent Framework Container 빌드 (Build Agent Framework Container)

### Container 구성

- **프레임워크**: Microsoft Agent Framework (Workflow Pattern)
- **포트**: 8000
- **엔드포인트**:
  - `/health` - Health check
  - `/` - Agent Framework 상태 정보
  - `/chat` - Workflow 실행 엔드포인트 (POST)

### Workflow Executors

- **Router Executor**: AI 기반 의도 분류 및 라우팅
- **Tool Executor**: MCP 도구 실행
- **Research Executor**: RAG 기반 지식 검색
- **General Executor**: 일반 대화
- **Orchestrator Executor**: 병렬 실행 및 결과 통합

### MCP 서버 제공 기능

- **실시간 날씨 정보**:
  - `get_weather(location)` - 전 세계 도시의 정확한 실시간 날씨 정보
  - 데이터 소스: wttr.in API (무료, 신뢰성 높음)
  - 지원 언어: 한글/영어 도시명 모두 가능 (예: 'Seoul', '서울')
  - 제공 정보: 온도, 체감온도, 날씨상태, 습도, 풍속/풍향, 관측시간

### 환경 변수 설정

`.env` 파일에 다음 변수들이 자동으로 설정됩니다:

- `AZURE_AI_PROJECT_ENDPOINT` - Azure AI Project 엔드포인트
- `AZURE_AI_MODEL_DEPLOYMENT_NAME` - 모델 배포 이름 (gpt-4o)
- `SEARCH_ENDPOINT`, `SEARCH_INDEX` - Azure AI Search 설정
- `MCP_ENDPOINT` - MCP Server 엔드포인트
- `APPLICATIONINSIGHTS_CONNECTION_STRING` - Application Insights (Analytics)
- `OTEL_*` - OpenTelemetry 설정 (Tracing)
- `AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED` - Prompt/Completion 기록

In [ ]:
# Container Registry 로그인
registry_name = CONTAINER_REGISTRY.split('.')[0]

print("=== Container Registry Login ===")
login_cmd = f"az acr login --name {registry_name}"
result = subprocess.run(login_cmd, shell=True, capture_output=True, text=True)

if result.returncode == 0:
    print(f"✅ Logged in to {registry_name}")
else:
    print(f"❌ Login failed: {result.stderr}")
print("="*50)

In [ ]:
# .env 파일 생성 (Agent Framework Container용)
print("=== Creating .env file for Agent Framework Container ===\n")

# 1. Application Insights Connection String 가져오기
print("📊 Getting Application Insights connection string...")
appinsights_cmd = f"""
az monitor app-insights component show \
    --resource-group {RESOURCE_GROUP} \
    --query "[0].connectionString" -o tsv
"""

result = subprocess.run(appinsights_cmd, shell=True, capture_output=True, text=True)

if result.returncode == 0 and result.stdout.strip():
    APP_INSIGHTS_CONN_STR = result.stdout.strip()
    print(f"✅ Application Insights connection string retrieved\n")
else:
    print(f"⚠️  Could not get Application Insights connection string")
    print(f"   Proceeding without Application Insights\n")
    APP_INSIGHTS_CONN_STR = ""

# 2. 모델 설정 가져오기 (config.json에서)
model_deployment_name = config.get("model_deployment_name", "gpt-4o")
model_version = config.get("model_version", "2024-11-20")
model_capacity = config.get("model_capacity", 50)
print(f"📦 Model Configuration:")
print(f"   Deployment Name: {model_deployment_name}")
print(f"   Model Version: {model_version}")
print(f"   Capacity (TPM): {model_capacity}")
print(f"   (from config.json - set in Lab 1 infrastructure deployment)\n")

# 3. .env 파일 생성
env_content = f"""# Azure AI Project Configuration (Microsoft Agent Framework)
AZURE_AI_PROJECT_ENDPOINT={simple_project_conn}

# Model Configuration
# The model deployment name and version from Azure OpenAI
# These values are automatically set from Lab 1 infrastructure deployment (infra/main.bicep)
AZURE_AI_MODEL_DEPLOYMENT_NAME={model_deployment_name}
AZURE_AI_MODEL_VERSION={model_version}

# MCP Server Configuration
MCP_ENDPOINT={MCP_ENDPOINT if MCP_ENDPOINT else ''}

# Azure AI Search Configuration (for Research Agent with RAG)
SEARCH_ENDPOINT={SEARCH_ENDPOINT}
SEARCH_INDEX={SEARCH_INDEX}
SEARCH_KEY={SEARCH_KEY}

# Application Insights Configuration (for Application Analytics)
APPLICATIONINSIGHTS_CONNECTION_STRING={APP_INSIGHTS_CONN_STR}

# OpenTelemetry Configuration (for Tracing)
OTEL_SERVICE_NAME=agent-framework-workflow
OTEL_TRACES_EXPORTER=azure_monitor
OTEL_METRICS_EXPORTER=azure_monitor
OTEL_LOGS_EXPORTER=azure_monitor
OTEL_PYTHON_LOGGING_AUTO_INSTRUMENTATION_ENABLED=true
AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED=true

# Masking / PII Handling
AGENT_MASKING_MODE=standard  # off|standard|strict
"""

env_file_path = "src/agent_framework/.env"

try:
    with open(env_file_path, 'w') as f:
        f.write(env_content)
    
    print(f"✅ Created {env_file_path}")
    print("\n📋 Environment variables:")
    for line in env_content.strip().split('\n'):
        if line and not line.startswith('#'):
            key = line.split('=')[0]
            print(f"   • {key}")
    
    print("\n💡 이 파일은 Docker 이미지에 포함됩니다.")
    
    if APP_INSIGHTS_CONN_STR:
        print("\n✅ Application Insights 설정 완료!")
        print("   → OpenTelemetry Tracing 활성화")
        print("   → GenAI content recording 활성화")
    else:
        print("\n⚠️  Application Insights 미설정")
        print("   → Analytics는 작동하지 않지만 Agent는 정상 작동합니다.")
    
    print(f"\n🔍 Azure AI Search 설정 완료!")
    print(f"   → RAG (Retrieval-Augmented Generation) 활성화")
    print(f"   → Research Agent가 지식 베이스 검색 가능")
    print(f"\n🤖 Model: {model_deployment_name} (version {model_version}, capacity {model_capacity}K TPM)")
    
except Exception as e:
    print(f"❌ Failed to create .env file: {e}")

print("\n" + "="*60)


In [ ]:
# Agent Framework Container 이미지 빌드 및 푸시
import time

framework_image = f"{CONTAINER_REGISTRY}/agent-framework:latest"

print("=== Building Agent Framework Image ===")
print(f"Image: {framework_image}\n")

# 빌드 (Azure Container Apps용 linux/amd64 플랫폼)
build_cmd = f"docker build --platform linux/amd64 -t {framework_image} ./src/agent_framework"
print("🔨 Building image (linux/amd64)...")
start_time = time.time()

result = subprocess.run(build_cmd, shell=True, capture_output=True, text=True)
elapsed = time.time() - start_time

if result.returncode == 0:
    print(f"✅ Build successful ({elapsed:.1f}s)")
    print(f"   Framework: Microsoft Agent Framework (Workflow Pattern)")
else:
    print(f"❌ Build failed: {result.stderr}")
    
# 푸시
if result.returncode == 0:
    print("\n📤 Pushing image to registry...")
    push_cmd = f"docker push {framework_image}"
    result = subprocess.run(push_cmd, shell=True, capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"✅ Push successful")
    else:
        print(f"❌ Push failed: {result.stderr}")

print("="*50)

## 5. Azure 리소스 확인 (Verify Azure Resources)

Agent Framework Service 배포 전에 필요한 Azure 리소스를 확인합니다.

**확인 항목:**
- ✅ Azure AI Project 리소스 ID

In [ ]:
# Azure AI Project 리소스 ID 확인
print("=== Verifying Azure Resources ===\n")

# URL에서 project_name 추출
if '/api/projects/' in simple_project_conn:
    project_name = simple_project_conn.split('/api/projects/')[-1].strip()
else:
    project_name = None

print(f"📋 Project Information:")
print(f"   Resource Group: {RESOURCE_GROUP}")
print(f"   Project Name: {project_name if project_name else 'Not found'}\n")

# AI Project 리소스 ID 가져오기
print("🔍 Finding AI Project resource...")
if project_name:
    ai_project_cmd = f"""
    az resource list \
        --resource-group {RESOURCE_GROUP} \
        --query "[?contains(name, '{project_name}') && type=='Microsoft.CognitiveServices/accounts/projects'].id" -o tsv
    """
else:
    ai_project_cmd = f"""
    az resource list \
        --resource-group {RESOURCE_GROUP} \
        --query "[?type=='Microsoft.CognitiveServices/accounts/projects'].id | [0]" -o tsv
    """

result = subprocess.run(ai_project_cmd, shell=True, capture_output=True, text=True)
if result.returncode == 0 and result.stdout.strip():
    ai_project_resource_id = result.stdout.strip()
    print(f"   ✅ AI Project Resource ID:")
    print(f"   {ai_project_resource_id}\n")
else:
    print(f"   ❌ Could not find AI Project")
    raise Exception("AI Project not found")

print("✅ All required resources verified!")
print("="*60)

## 6. Agent Framework Service 배포 (Deploy Agent Framework Service)

Agent Framework Service를 배포하고 **배포 직후 자동으로** Managed Identity를 구성합니다.

### 자동 수행 작업

1. ✅ Container App 배포 (replicas=0으로 시작)
2. ✅ System-assigned Managed Identity 활성화
3. ✅ Azure AI User 역할 할당 (AI Project scope)
4. ✅ 권한 전파 대기 안내

> 💡 **중요**: 배포와 권한 설정을 한 번에 처리하므로 완료까지 약 3-4분 소요됩니다.

In [ ]:
# Agent Framework Service를 Container App으로 배포 + Managed Identity 권한 설정
framework_app_name = "agent-framework"

print("=== Deploying Agent Framework Service to ACA ===")
print(f"App Name: {framework_app_name}\n")

print("💡 환경 변수는 이미 Docker 이미지에 포함되어 있습니다.\n")

# 1. Container App 배포 (Managed Identity 포함, replicas=0)
deploy_cmd = f"""
az containerapp create \
    --name {framework_app_name} \
    --resource-group {RESOURCE_GROUP} \
    --environment {CONTAINER_ENV_ID.split('/')[-1]} \
    --image {framework_image} \
    --target-port 8000 \
    --ingress external \
    --min-replicas 0 \
    --max-replicas 3 \
    --cpu 1.0 \
    --memory 2.0Gi \
    --registry-server {CONTAINER_REGISTRY} \
    --system-assigned \
"""

print("🚀 Deploying Agent Framework Service with Managed Identity...")
print("   (Starting with 0 replicas to configure permissions first)")
result = subprocess.run(deploy_cmd, shell=True, capture_output=True, text=True, timeout=180)

if result.returncode == 0:
    print("✅ Deployment successful\n")
    
    # Get endpoint
    show_cmd = f"""
    az containerapp show \
        --name {framework_app_name} \
        --resource-group {RESOURCE_GROUP} \
        --query properties.configuration.ingress.fqdn -o tsv
    """
    result = subprocess.run(show_cmd, shell=True, capture_output=True, text=True)
    FRAMEWORK_ENDPOINT = f"https://{result.stdout.strip()}"
    
    print(f"🌐 Agent Framework Endpoint: {FRAMEWORK_ENDPOINT}")
    
    # Update config
    config['agent_framework_endpoint'] = FRAMEWORK_ENDPOINT
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)
    print("✅ Config updated\n")
    
    # 2. Managed Identity Principal ID 가져오기
    print("="*60)
    print("🔐 Configuring Permissions\n")
    
    print("1️⃣ Getting Managed Identity Principal ID...")
    identity_cmd = f"""
    az containerapp show \
        --name {framework_app_name} \
        --resource-group {RESOURCE_GROUP} \
        --query identity.principalId -o tsv
    """
    
    result = subprocess.run(identity_cmd, shell=True, capture_output=True, text=True)
    if result.returncode == 0 and result.stdout.strip():
        principal_id = result.stdout.strip()
        print(f"   ✅ Principal ID: {principal_id}\n")
    else:
        print(f"   ❌ Failed to get Principal ID\n")
        raise Exception("Failed to get Managed Identity Principal ID")
    
    # 3. Azure AI User 역할 할당
    print("2️⃣ Assigning 'Azure AI User' role to AI Project...")
    print(f"   Scope: {ai_project_resource_id}")
    role_assignment_cmd = f"""
    az role assignment create \
        --assignee {principal_id} \
        --role "Azure AI User" \
        --scope {ai_project_resource_id}
    """
    
    result = subprocess.run(role_assignment_cmd, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print("   ✅ Azure AI User role assigned\n")
    elif "already exists" in result.stderr.lower():
        print("   ✅ Azure AI User role already exists\n")
    else:
        print(f"   ❌ Role assignment FAILED!")
        print(f"   Error: {result.stderr}\n")
    
    # 4. 권한 검증
    print("3️⃣ Verifying role assignments...\n")
    import time
    time.sleep(5)
    
    role_check_cmd = f"""
    az role assignment list \
        --assignee {principal_id} \
        --query "[].{{role:roleDefinitionName, scope:scope}}" -o json
    """
    
    result = subprocess.run(role_check_cmd, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        import json as json_lib
        current_roles = json_lib.loads(result.stdout)
        
        print(f"   📋 Current Role Assignments ({len(current_roles)} total):\n")
        
        required_roles = {"Azure AI User (AI Project)": False}
        
        for role in current_roles:
            scope_parts = role['scope'].split('/')
            resource_name = scope_parts[-1] if scope_parts else 'Unknown'
            role_name = role['role']
            
            print(f"      • {role_name} → {resource_name}")
            
            if role_name == "Azure AI User":
                if "projects" in role['scope'] or ai_project_resource_id in role['scope']:
                    required_roles["Azure AI User (AI Project)"] = True
        
        print(f"\n   🔍 Required Roles Verification:")
        all_roles_ok = True
        for role_name, assigned in required_roles.items():
            status = "✅" if assigned else "❌"
            print(f"      {status} {role_name}")
            if not assigned:
                all_roles_ok = False
        
        if all_roles_ok:
            print(f"\n   ✅ All required roles are assigned!\n")
        else:
            print(f"\n   ❌ Some required roles are missing!\n")
    
    # 5. 권한 전파 대기 안내
    print("="*60)
    print("4️⃣ Permissions assigned - waiting for propagation...\n")
    print("⚠️  Azure RBAC 권한 전파는 최대 5-10분 소요될 수 있습니다.")
    print("   Container는 replicas=0 상태로 유지됩니다.\n")
    
    print("📋 다음 단계:")
    print("   1. 위의 'Required Roles Verification'이 모두 ✅인지 확인")
    print("   2. 2-3분 정도 기다리세요")
    print("   3. 다음 셀(섹션 5)을 실행하여 Container를 시작하세요")
    
    print("\n" + "="*60)
    print("✅ Permissions configured successfully!")
    print(f"\n🌐 Endpoint: {FRAMEWORK_ENDPOINT}")
    print(f"\n⏳ 권한 전파를 기다린 후 다음 셀을 실행하세요!")
else:
    print(f"❌ Deployment failed: {result.stderr}")
    FRAMEWORK_ENDPOINT = None

print("\n" + "="*60)

## 7. Agent Framework Service 시작 (Start Service)

권한 전파가 완료된 후 이 셀을 실행하여 Container를 시작합니다.

**실행 시점:**
- ⏰ 섹션 4 완료 후 **2-3분 대기**
- ⚠️ 권한 오류 발생 시: 추가로 2-3분 더 기다린 후 재실행

In [ ]:
# replicas를 1로 확장
scale_cmd = f"""
az containerapp update \
    --name {framework_app_name} \
    --resource-group {RESOURCE_GROUP} \
    --min-replicas 1 \
    --max-replicas 1
"""

print("🚀 Scaling to 1 replica...")
result = subprocess.run(scale_cmd, shell=True, capture_output=True, text=True, timeout=120)

if result.returncode == 0:
    print("✅ Agent Framework Service started successfully!")
    print(f"\n🌐 Endpoint: {FRAMEWORK_ENDPOINT}")
    print("\n💡 Container가 시작되는 데 약 30초 정도 소요됩니다.")
    print(f"   로그 확인: az containerapp logs show --name {framework_app_name} --resource-group {RESOURCE_GROUP} --tail 50")
else:
    print(f"❌ Failed to start: {result.stderr}")

print("\n" + "="*60)

## 8. 배포된 Agent Framework 테스트

**✅ Foundry Agent vs Agent Framework - 모니터링 & 트레이싱:**

**Foundry Agent (Lab 3):**
- ✅ Application Analytics 자동 수집
- ✅ Tracing 자동 활성화 (Azure Agent Service 기본 기능)
- ✅ AI Foundry 포털에서 실행 흐름 확인 가능

**Agent Framework (Lab 4 - 현재):**
- ✅ **Application Analytics 수집 가능** (OpenTelemetry 구현됨)
- ✅ **Tracing 활성화됨** (커스텀 계측 코드 구현)
- ✅ **AI Foundry 포털에서 실행 흐름 확인 가능**
- 📊 **추가 구현된 기능:**
  - Router 의도 분류 추적
  - Executor별 실행 시간 측정
  - Tool Agent MCP 호출 상세 추적
  - Research Agent RAG 검색 추적
  - Orchestrator 병렬 실행 추적

> 💡 **Agent Framework의 장점**:  
> 코드 중심 접근으로 **완전한 커스터마이징**이 가능하며,  
> OpenTelemetry를 직접 구현하여 **세밀한 모니터링**을 제공합니다.

**트레이싱 계층 구조:**
```
📊 api.chat (HTTP Request)
  └─ 🤖 agent_framework.workflow
      ├─ 🧭 workflow.router (Intent Classification)
      │   ├─ router.method: rule_based / ai_based
      │   └─ router.intent: tool / research / orchestrator / general
      │
      └─ ⚙️ workflow.executor.{type}
          ├─ tool_agent.execute → mcp_call
          ├─ research_agent.execute → search + generate
          ├─ general_agent.execute
          └─ orchestrator.parallel_execution
```

**Workflow 실행 흐름:**
1. Router Executor: 의도 분류 (tool/research/orchestrator/general)
2. 해당 Executor로 메시지 라우팅
3. Workflow Context를 통한 출력 수집
4. 통합된 응답 반환
5. **모든 단계가 Azure AI Foundry Tracing에 기록됨**

In [ ]:
import requests
import json

print("=== 배포된 Agent Framework Service 테스트 ===\n")

if not FRAMEWORK_ENDPOINT:
    print("❌ FRAMEWORK_ENDPOINT가 설정되지 않았습니다!")
    print("   섹션 4를 먼저 실행하세요.\n")
else:
    print(f"🌐 Agent Framework Endpoint: {FRAMEWORK_ENDPOINT}\n")
    
    # 1. Health check
    print("1️⃣ Health Check:")
    try:
        response = requests.get(f"{FRAMEWORK_ENDPOINT}/health", timeout=10)
        if response.status_code == 200:
            print(f"   ✅ Health: {response.json()}\n")
        else:
            print(f"   ❌ Health check failed: {response.status_code}\n")
    except Exception as e:
        print(f"   ❌ Error: {e}\n")
    
    # 2. Root endpoint 확인
    print("2️⃣ Agent Framework Status:")
    try:
        response = requests.get(f"{FRAMEWORK_ENDPOINT}/", timeout=10)
        if response.status_code == 200:
            status = response.json()
            print(f"   ✅ Status: {status.get('status')}")
            print(f"   📋 Framework: {status.get('framework')}")
            print(f"   🔧 Agents:")
            for agent_name, available in status.get('agents', {}).items():
                icon = "✅" if available else "❌"
                print(f"      {icon} {agent_name}: {available}")
            print()
        else:
            print(f"   ❌ Status check failed: {response.status_code}\n")
    except Exception as e:
        print(f"   ❌ Error: {e}\n")
    
    print("="*70)
    print("\n💡 Agent Framework Service가 정상 작동 중입니다!")
    print("   다음 셀에서 Workflow 실행을 테스트하세요.\n")
    print("   MCP 서버는 wttr.in API를 사용하여 실시간 날씨 정보를 제공합니다.\n")
    print("="*70)

## 9. Workflow Pattern 테스트 (다양한 질문)

배포된 Agent Framework에 다양한 질문을 보내서 Workflow 실행을 테스트합니다.

**Router 의도 분류:**
- **tool**: 단순 도구 실행 (날씨)
- **research**: 단순 지식 검색 (RAG)
- **orchestrator**: 복합 질의 (도구 + 지식)
- **general**: 일반 대화

---

### 📚 Research Agent의 Citation 기능

Research Agent가 Azure AI Search를 사용하여 답변할 때, **자동으로 출처(citation)**를 표시합니다:

**Citation 형식:**
- `【1:0†source】` = 첫 번째 검색 결과 문서
- `【2:0†source】` = 두 번째 검색 결과 문서
- `【3:0†source】` = 세 번째 검색 결과 문서

**예시 응답:**
```
📚 [RAG-based Answer]

RAG 패턴은 검색 증강 생성 방식입니다【1:0†source】.
주요 장점은 정확도 향상과 환각 감소입니다【2:0†source】.
Azure AI Search와 통합하여 하이브리드 검색을 수행할 수 있습니다【1:0†source】【3:0†source】.
```

**작동 원리:**
1. Research Agent가 Azure AI Search로 지식 베이스 검색
2. 상위 5개 검색 결과를 LLM context에 주입 ([Document 1], [Document 2] 형식)
3. LLM에게 `【N:0†source】` 형식으로 출처 인용하도록 프롬프트 지시
4. LLM이 답변 생성 시 관련 문서를 자동으로 인용
5. Citation은 답변 텍스트에 자연스럽게 포함됨

> 💡 **Lab 3과의 차이:** Lab 3은 Azure AI Foundry의 `AzureAISearchTool`이 자동으로 citation을 생성하며, Lab 4는 동일한 형식(`【N:0†source】`)을 커스텀 코드로 구현하여 일관된 사용자 경험을 제공합니다.

In [ ]:
# Agent Framework Workflow 테스트
import requests
import json
import time
import statistics

print("=== Agent Framework Workflow 테스트 ===\n")

# 테스트 케이스 (3개)
test_cases = [
    {
        "message": "안녕하세요",
        "description": "General Executor (일반 대화) - Warmup"
    },
    {
        "message": "서울의 현재 날씨를 알려주세요. 온도와 체감온도, 날씨 상태, 습도, 바람 정보를 모두 포함해주세요.",
        "description": "Tool Executor (날씨 조회 - wttr.in API)"
    },
    {
        "message": "제주도 여행 추천 명소를 알려주세요",
        "description": "Research Executor (지식 검색 - 제주도 여행)"
    }
]

success_count = 0
fail_count = 0
latencies = []

for i, test in enumerate(test_cases, 1):
    print(f"{'='*70}")
    print(f"[Test {i}/{len(test_cases)}]")
    print(f"질문: {test['message']}")
    print(f"예상 경로: {test['description']}")
    print(f"{'='*70}")
    
    try:
        start_req = time.perf_counter()
        response = requests.post(
            f"{FRAMEWORK_ENDPOINT}/chat",
            json={"message": test['message']},
            headers={"Content-Type": "application/json"},
            timeout=90
        )
        elapsed_req = (time.perf_counter() - start_req) * 1000
        
        if response.status_code == 200:
            result = response.json()
            full_resp = result.get('response', 'No response') or ''
            print(f"\n✅ 응답 성공 (HTTP {response.status_code}) - {elapsed_req:.1f} ms")
            latencies.append(elapsed_req)
            print(f"\n📝 Workflow 응답:")
            print(full_resp)
            print()
            success_count += 1
        else:
            print(f"\n❌ 요청 실패 (HTTP {response.status_code})")
            print(f"오류: {response.text[:200]}")
            print()
            fail_count += 1
        
    except Exception as e:
        print(f"\n❌ 오류 발생: {e}")
        print()
        fail_count += 1
    
    if i < len(test_cases):
        time.sleep(2)

print("="*70)
print(f"\n📊 테스트 결과:")
print(f"   ✅ 성공: {success_count}/{len(test_cases)}")
print(f"   ❌ 실패: {fail_count}/{len(test_cases)}")
print(f"\n💡 날씨 정보는 wttr.in API에서 실시간으로 제공됩니다.")


## 10. 정리 및 비교 (Summary & Comparison)

### Foundry Agent vs Agent Framework 비교

#### 아키텍처 패턴

**Foundry Agent (Lab 3):**
```
Main Agent
  └─> handoff_to_tool_agent()
        └─> Tool Agent executes
              └─> returns to Main Agent
  └─> handoff_to_research_agent()
        └─> Research Agent executes
              └─> returns to Main Agent
```

**Agent Framework (Lab 4):**
```
Router Executor
  └─> send_message(target="tool")
        └─> Tool Executor executes
              └─> yield_output()
  └─> send_message(target="orchestrator")
        └─> Orchestrator runs Tool + Research in parallel
              └─> asyncio.gather()
                    └─> yield_output(combined)
```

#### 장단점 비교

| 특성 | Foundry Agent | Agent Framework |
|------|--------------|----------------|
| **관리형 서비스** | ✅ Azure 완전 관리 | ❌ 코드 직접 관리 |
| **Thread 상태** | ✅ 자동 유지 | ❌ 수동 관리 |
| **병렬 실행** | ❌ Handoff는 순차적 | ✅ asyncio.gather() |
| **유연성** | ⚠️ Azure API 제약 | ✅ 완전한 제어 |
| **로컬 개발** | ⚠️ Azure 의존성 | ✅ 독립 실행 가능 |
| **프로덕션 준비** | ✅ 엔터프라이즈급 | ⚠️ 추가 구현 필요 |

#### 사용 권장 시나리오

**Foundry Agent 선택:**
- Azure 생태계에 깊이 통합된 솔루션
- 엔터프라이즈급 관리형 서비스 필요
- Thread 기반 대화 컨텍스트 자동 관리
- 빠른 프로토타이핑과 배포

**Agent Framework 선택:**
- 세밀한 실행 흐름 제어 필요
- 복잡한 병렬 처리 패턴
- 커스텀 오케스트레이션 로직
- 로컬 개발 및 빠른 반복

### 실습 완료! 🎉

**학습 내용 요약:**
1. ✅ Microsoft Agent Framework Workflow Pattern 이해
2. ✅ Executor 기반 Multi-Agent 구현 및 배포
3. ✅ Workflow Context 메시지 라우팅
4. ✅ Parallel execution (Orchestrator) 패턴
5. ✅ Foundry Agent와 Framework 비교 분석

**다음 단계:**
- Application Analytics 및 Tracing 데이터 분석
- 커스텀 Executor 추가 실험
- 두 패턴의 성능 및 비용 비교
- 프로덕션 환경 적용 고려사항 검토